## Programming Excersice 2: Logistic Regression
#### Author - Rishabh Jain

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
%matplotlib inline

### 1&nbsp;&nbsp;&nbsp;&nbsp;Logistic Regression
In this part of the exercise, we will build a logistic regression model to predict whether the student gets admitted into a university.  

Suppose that we are the adminstrator of a department and want to determine each applicant's chance of admission based on their results in two exams. We have hisotrical data from previous applicants that we can use as a training set for logistic regression. For each applicant, we have their scores in two exams and their admission decision.

Our task is to build a classification model that estimates an applicant's probability of admission based on the score from those two exams.